In [1]:
# Header Files
import io
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import sys
import random
import warnings

import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2

#from tqdm import tqdm 

from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
#from keras.models import model_from_json

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.35)
config=tf.compat.v1.ConfigProto(gpu_options=gpu_options)
config.log_device_placement = False  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

In [3]:
# Path Variables
INPUT_H5 = "data_images.h5"
MASKS_H5 = "data_masks.h5"
MODEL_H5 = "building_weights_FineTune.h5"
MODEL_FINAL = "building_weights_FINETUNE_FINAL.h5"

# Size Variables

IMG_SIZE = (256,256)
BATCH_SIZE = 64
EPOCHS = 3
SEED = 303

In [4]:
def getIOU(y_true, y_pred, smooth=1):
  tf.cast(y_true, tf.float32)
  tf.cast(y_pred, tf.float32)
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  return K.mean((intersection + smooth) / (union + smooth), axis=0)

In [5]:
hf1 = h5py.File(INPUT_H5, 'r')
hf2 = h5py.File(MASKS_H5,'r')

input_image=[]
input_mask=[]

input_image = list(hf1.keys())
input_image = sorted(input_image)
#print(input_image)

input_mask = list(hf2.keys())
input_mask  = sorted(input_mask)
#print(input_mask)

print(len(input_image))
print(len(input_mask))

hf1.close()
hf2.close()

280741
280741


In [6]:
val_samples = 28074
random.Random(1337).shuffle(input_image)
random.Random(1337).shuffle(input_mask)

train_input_img_paths  =  input_image[:-val_samples]
train_target_img_paths =  input_mask[:-val_samples]
val_input_img_paths    =  input_image[-val_samples:]
val_target_img_paths   =  input_mask[-val_samples:]

train_input_img_paths=sorted(train_input_img_paths)
train_target_img_paths=sorted(train_target_img_paths)
val_input_img_paths=sorted(val_input_img_paths)
val_target_img_paths=sorted(val_target_img_paths)

In [7]:
class DataLoad(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x= np.zeros((self.batch_size, 256,256,3), dtype=np.uint8)
        for j, path in enumerate(batch_input_img_paths):
            h5 = h5py.File(INPUT_H5, 'r')
            data=h5.get(path)
            data=np.array(data)
            img = Image.open(io.BytesIO(data))
            img=np.array(img)
            img=resize(img, (256,256), mode='constant',  preserve_range=True)
            x[j] = img
        
        y=np.zeros((self.batch_size, 256, 256, 1), dtype=np.bool)
        for j, path in enumerate(batch_target_img_paths):
            mask = np.zeros((256,256, 1), dtype=np.bool)
            h51=h5py.File(MASKS_H5,'r')
            data=h51.get(path)
            data=np.array(data)
            mask_=Image.open(io.BytesIO(data))
            
            mask_=np.array(mask_)
            mask_ = np.expand_dims(resize(mask_, (256,256), mode='constant',preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_) 
            y[j]=mask
            
           
        return x, y
    




train_gen = DataLoad(BATCH_SIZE, IMG_SIZE, train_input_img_paths, train_target_img_paths)
val_gen = DataLoad(BATCH_SIZE, IMG_SIZE, val_input_img_paths, val_target_img_paths)

print(len(train_gen))
print(len(val_gen))

3947
438


In [8]:
callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(MODEL_H5, verbose=1, save_best_only=True)]

In [9]:
model = load_model("building_weights_RESNET.h5",custom_objects={'getIOU':getIOU})
print('Model Loaded')



Model Loaded


In [10]:
# Uncomment line below when fine tuning
model.trainable = True

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',getIOU])



In [11]:
results = model.fit(train_gen,epochs=EPOCHS,batch_size=BATCH_SIZE, validation_data=val_gen,callbacks=callbacks)
print(results)

model.save(MODEL_FINAL)
print("Model Saved")

Epoch 1/3


ResourceExhaustedError:  OOM when allocating tensor with shape[2048,1024,3,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node UnetResnet/conv2d_transpose/conv2d_transpose (defined at <ipython-input-11-70a22609479a>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_39350]

Function call stack:
train_function


In [4]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime


%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 29764), started 0:02:32 ago. (Use '!kill 29764' to kill it.)